In [1]:
import tensorflow as tf
import torch

print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("PyTorch CUDA Version:", torch.version.cuda)
print("Is CUDA available:", torch.cuda.is_available())


TensorFlow version: 2.10.1
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
PyTorch CUDA Version: 11.8
Is CUDA available: True


In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
import numpy as np

c:\Users\yoges\.conda\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [4]:
text = "i am not liking this class"


In [5]:
input = tokenizer(text, return_tensors='tf')

In [6]:
output = model(**input)

In [7]:
prob = tf.nn.softmax(output.logits,axis=-1).numpy()[0]

In [8]:
prob

array([9.9967754e-01, 3.2253243e-04], dtype=float32)

In [9]:
predict_class_id = np.argmax(prob)
model.config.id2label[predict_class_id]

'NEGATIVE'

In [10]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))



Device set to use cuda:0


[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [12]:
model_name = "facebook/nllb-200-distilled-600M"

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [14]:
input_text = "this is just a test"
tokenizer.src_lang = "eng_Latn"
encoded = tokenizer(input_text,return_tensors="pt")


In [15]:
output = model.generate(**encoded,forced_bos_token_id = tokenizer.convert_tokens_to_ids("tam_Taml"))

In [16]:
output

tensor([[     2, 256170,  16399,   7219,  89389, 135478,  67020,      2]])

In [17]:
tokenizer.batch_decode(output,skip_special_tokens=True)[0]

'இது ஒரு சோதனை மட்டுமே'

#### Finetuning

In [18]:
from datasets import load_dataset
dataset = load_dataset("imdb")

In [19]:
dataset["train"][1000]

{'text': 'Although I have to admit I laughed more watching this movie than the last few comedies I saw.<br /><br />The budget must have consisted of pocket change from the actors. The production values are so low that they actual made it kind of fun to watch. Reminds me of the Robot Monster made up of a guy in a gorilla suit with a cardboard diving helmet on.<br /><br />In one scene a hapless victim gets their arm and leg cut off. Geez, hard to believe but the Black Knight scene from Holy Grail was more realistic. I kept wondering why the victim didn\'t start shouting " None Shall Pass" and " It\'s only a flesh wound, I\'ve had worse". It was one of the funniest scenes I\'ve seen in the past year.<br /><br />The "gladiator/demon" was a stitch too. Between the horribly cheap costume and the geeky look of the guy in it the end result was hysterical.<br /><br />Truly a movie that is bad enough to be watchable. Kind of like seeing a slow motion auto accident on film.<br /><br />',
 'label'

In [20]:
from transformers import AutoTokenizer
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
def tokenizer_fn(data):
    return tokenizer(data['text'],padding="max_length",truncation=True)

tokenized_dataset = dataset.map(tokenizer_fn,batched=True)

In [21]:
tokenized_dataset 

# Here input_ids are basically numerical representation of texts so we'll remove the 'text'

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [22]:
len(tokenized_dataset["train"][0]['attention_mask'])

512

In [23]:
len(tokenized_dataset["train"][0]['text'].split())

288

In [24]:
tokenized_dataset = tokenized_dataset.remove_columns(['text'])

In [25]:
tokenized_dataset = tokenized_dataset.rename_column("label","labels")

In [26]:
tokenized_dataset.set_format('torch')

In [27]:
from transformers import AutoModelForSequenceClassification

In [28]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,num_labels = 2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
from transformers import TrainingArguments, Trainer

In [32]:
training_args = TrainingArguments(
    output_dir="./result",
    save_strategy='epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01
)

In [33]:
trainer = Trainer(model=model,
        args = training_args,
        train_dataset=tokenized_dataset['train'].select(range(100)),
        eval_dataset=tokenized_dataset['test'].select(range(100))
        )
trainer.train()

Step,Training Loss


TrainOutput(global_step=13, training_loss=0.4399282748882587, metrics={'train_runtime': 173.8504, 'train_samples_per_second': 0.575, 'train_steps_per_second': 0.075, 'total_flos': 13246739865600.0, 'train_loss': 0.4399282748882587, 'epoch': 1.0})